In [4]:
import arcpy
import arcgis
import pandas as pd
import os

In [39]:
# Read and prep data
# Read
file = r"C:\Users\jneuj\Dropbox\2. research projects\4. Development\1. ABA\2. ABA Sebha 2019\4. Tools\Tools Draft\Service areas\REACH_LBY_ABA_MFGD_Service Areas_v1_JNE.xlsx"
sheet = "Electricity KII"
quest = pd.read_excel(file, sheet_name = "Electricity KII ")

# Filter first rows of generic data out
quest = quest[quest['Sub-research question '].notnull() & quest['Geometry'].notnull()]


In [40]:
quest

,Research Question,Sub-research question,Question #,Indicator/variable,Category,Question,Probes,Facilitator Instructions/Training,Geometry,Attributes,Comments Joost
10,1. Where are the city's neighbourhoods and wha...,1.1. What key infrastructure exists in eac...,11,NaN,Service provider,Where are the GECOL offices in Sebha?,NaN,Draw points on map,Point,"Function, amount of employees, operational",NaN
11,1. Where are the city's neighbourhoods and wha...,1.1. What key infrastructure exists in eac...,12,NaN,Service area,Can you indicate how the electricity grid in S...,Are their maps available showing the electric...,Two options: drawing all the lines (very time ...,Line,"Voltage, operational",The 'electricity grid' we will not have. We on...
13,1. Where are the city's neighbourhoods and wha...,1.1. What key infrastructure exists in eac...,14,NaN,Service area,Where does the electricity supply for Sebha co...,NaN,"If applicable, draw the high voltage supply li...",Point,?,NaN
14,1. Where are the city's neighbourhoods and wha...,1.1. What key infrastructure exists in eac...,15,NaN,Service area,How has the electricity grid developed over th...,Are their maps available demonstrating the dev...,"If there are no existing maps, please draw zon...",Polygon,year,NaN
15,1. Where are the city's neighbourhoods and wha...,1.1. What key infrastructure exists in eac...,16,NaN,Service area,Is the electricity grid laid out above the gro...,NaN,Draw zones on the map. Use the words 'above gr...,Polygon,Above/Below ground,NaN
16,1. Where are the city's neighbourhoods and wha...,1.1. What key infrastructure exists in eac...,17,NaN,Service area,Where are the transformers?,NaN,Put points indicating exact location of transf...,Point,Technical details on transformer?,NaN
18,2. What are the characteristics of service del...,2.1. Which DCUs and groups within them hav...,19,NaN,Service area,Which neighbourhoods are not connected to the ...,Or: which areas are connected but have limited...,Indicate areas that are not connected to the e...,Polygon,Reason,NaN
20,2. What are the characteristics of service del...,2.1. Which DCUs and groups within them hav...,21,NaN,Service quality,Where do power outages most frequently occur?,Reason? Failure of grid? Rolling blackouts?,Indicate polygons on map,Polygon,"Reason, frequency (hours/day or days/week with...",NaN
21,2. What are the characteristics of service del...,2.1. Which DCUs and groups within them hav...,22,NaN,Service quality,What causes the power outages?,NaN,"If causes are spatially different, make sure t...",Polygon,Cause,NaN
22,2. What are the characteristics of service del...,2.1. Which DCUs and groups within them hav...,23,NaN,Service quality,What are the main consequences of the power ou...,What are the consequences for households? For ...,"If consequences are spatially different, make ...",Polygon,Consequence,NaN


In [32]:
# Create empty file geodatabase for each FGD/KI
fgdb_loc = r"C:\Users\jneuj\Dropbox\6. GIS\GeoData\ABA Sebha"
arcpy.CreateFileGDB_management(fgdb_loc, sheet)

SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

<Result 'C:\\Users\\jneuj\\Dropbox\\6. GIS\\GeoData\\ABA Sebha\\Electricity KII.gdb'>

In [ ]:
for q, in quest.Question:
    out_path = os.path.join(fgdb_loc, sheet + ".gdb")
    out_name = "q" + str'
    alias = q
    geometry_type = quest.Geometry
    sr = arcpy.SpatialReference(3857)
    
    # Create feature class for each question
    arcpy.CreateFeatureclass_management(out_path, out_name, geometry_type, spatial_reference = sr )
    
    

In [36]:
print(os.path.join(fgdb_loc, sheet + ".gdb"))

C:\Users\jneuj\Dropbox\6. GIS\GeoData\ABA Sebha\Electricity KII.gdb


In [44]:
out_path = os.path.join(fgdb_loc, sheet + ".gdb")
out_name = "q" + str(11)  #Can later out a coded name for the question
alias = "Where are the GECOL offices in Sebha? "
geometry_type = "Point"
sr = arcpy.SpatialReference(3857)


arcpy.CreateFeatureclass_management(out_path, out_name, geometry_type, spatial_reference = sr, out_alias = alias)

<Result 'C:\\Users\\jneuj\\Dropbox\\6. GIS\\GeoData\\ABA Sebha\\Electricity KII.gdb\\q11'>

In [46]:
for q in quest:
    print(quest.)
    
    

    
    

ValueError: too many values to unpack (expected 2)

In [47]:
q

'If unsatisfied with the quality of health services, who can citizens go to in order to complain? '

In [48]:
quest.q

AttributeError: 'DataFrame' object has no attribute 'q'